In [1]:
import numpy
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import keras.initializers
from keras import Model

Using TensorFlow backend.


In [2]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


In [3]:
# load dataset
dataframe = pandas.read_csv("sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

In [5]:
# convert the output from sring representation to number
le = LabelEncoder()
encoded_Y=le.fit_transform(Y)

In [5]:
# create a baseline model
def create_baseline():
    # create model, write code below
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    # Compile model, write code below
    model.compile(loss='binary_crossentropy',optimizer='adam', decay=0.0, amsgrad=False),metrics=['acc'])
    return model
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Results: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Results: 80.78% (5.35%)


In [6]:
# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Standardized: 84.63% (7.65%)


In [7]:
# Now testing on a smaller model
def create_smaller():
# create model, write code below
    model = Sequential()
    model.add(Dense(30, activation='relu', input_shape=(60,), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    # Compile model, write code below
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
    return model

    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Smaller: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Smaller: 85.04% (7.38%)


In [15]:
# smaller model
def create_larger():
# create model, write code below
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Dense(30, activation='relu', kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    # Compile model, write code below
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_larger, epochs=100, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 84.09% (7.55%)


In [11]:
# smaller model
def create_optimum():
# create model, write code below
    model = Sequential()
    model.add(Dense(60, activation='relu', input_shape=(60,), kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Dense(30, activation='relu',kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    model.add(Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None)))
    # Compile model, write code below
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_optimum, epochs=100, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Larger: 85.09% (8.49%)


In [26]:
#using functional API to acheive the same
from keras import Input
def create_functional():
    input_tensor = Input(shape=(60,))
    x = layers.Dense(30, activation='relu')(input_tensor)
    output_tensor = layers.Dense(1, activation='sigmoid')(x)
    model = Model(input_tensor, output_tensor)
    model.compile(loss='binary_crossentropy',optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),metrics=['acc'])
    return model
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_functional, epochs=200, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 85.97% (7.75%)


In [11]:
import tensorflow as tf

class MyModel(tf.keras.Model):

    def __init__(self):
        super(MyModel, self).__init__()
        self.dense1 = tf.keras.layers.Dense(60, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

def get_model():
    model = MyModel()
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['acc'])
    return model

estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=get_model, epochs=10, batch_size=50, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Larger: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Larger: 78.76% (7.17%)
